In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

2024-08-04 14:23:41.391253: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-04 14:23:41.391355: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-04 14:23:41.520665: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
!pip install opendatasets
!pip install pandas

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [3]:
import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/datasets/apollo2506/landuse-scene-classification")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  ayushvikasdaga


Your Kaggle Key:

  ································


Dataset URL: https://www.kaggle.com/datasets/apollo2506/landuse-scene-classification


100%|██████████| 1.98G/1.98G [01:12<00:00, 29.6MB/s]


In [4]:
! pip install torch torchvision

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
import numpy as np
import random
import tensorflow as tf

# Set seeds for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

# If using torch, set seed for it as well
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [7]:
import keras
from keras.applications.inception_v3 import InceptionV3
input_shape = (299, 299, 3)
base_model = InceptionV3(input_shape=input_shape, weights='imagenet', include_top=False)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [8]:
base_model.trainable = True

In [9]:
from keras.optimizers import Adam

# Create the optimizer with the desired parameters
optimizer = Adam(learning_rate=0.00005)
num_classes = 21
inputs = keras.Input(shape=input_shape)
x = keras.layers.GlobalAveragePooling2D(name='avg_pool')(base_model(inputs))
x = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(x)
model = keras.Model(inputs, x)

In [10]:
from tensorflow.keras import backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

In [11]:
model.compile( optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', 'recall', 'precision', f1_score])

In [12]:
import pandas as pd
train_csv = pd.read_csv("/kaggle/working/landuse-scene-classification/train.csv")
train_images = train_csv["Filename"]
train_labels = train_csv["Label"]
train_csv

,Unnamed: 0,Filename,Label,ClassName
0,5818,runway/runway_000259.png,16,runway
1,1327,intersection/intersection_000348.png,10,intersection
2,2529,agricultural/agricultural_000025.png,0,agricultural
3,3865,chaparral/chaparral_000195.png,5,chaparral
4,2024,airplane/airplane_000260.png,1,airplane
...,...,...,...,...
7345,779,mobilehomepark/mobilehomepark_000090.png,12,mobilehomepark
7346,6148,parkinglot/parkinglot_000386.png,14,parkinglot
7347,1453,freeway/freeway_000235.png,8,freeway
7348,5252,baseballdiamond/baseballdiamond_000144.png,2,baseballdiamond


In [13]:
import pandas as pd
test_csv = pd.read_csv("/kaggle/working/landuse-scene-classification/test.csv")
test_images = test_csv["Filename"]
test_labels = test_csv["Label"]
test_csv

,Unnamed: 0,Filename,Label,ClassName
0,311,river/river_000149.png,15,river
1,975,mediumresidential/mediumresidential_000335.png,11,mediumresidential
2,547,overpass/overpass_000338.png,13,overpass
3,670,storagetanks/storagetanks_000307.png,18,storagetanks
4,330,river/river_000370.png,15,river
...,...,...,...,...
1045,588,chaparral/chaparral_000471.png,5,chaparral
1046,316,river/river_000313.png,15,river
1047,1012,denseresidential/denseresidential_000335.png,6,denseresidential
1048,855,parkinglot/parkinglot_000414.png,14,parkinglot


In [14]:
import pandas as pd
validation_csv = pd.read_csv("/kaggle/working/landuse-scene-classification/validation.csv")
validation_images = validation_csv["Filename"]
validation_labels = validation_csv["Label"]
validation_csv

,Unnamed: 0,Filename,Label,ClassName
0,1928,mediumresidential/mediumresidential_000305.png,11,mediumresidential
1,102,buildings/buildings_000191.png,4,buildings
2,823,tenniscourt/tenniscourt_000224.png,19,tenniscourt
3,2083,denseresidential/denseresidential_000436.png,6,denseresidential
4,1579,baseballdiamond/baseballdiamond_000280.png,2,baseballdiamond
...,...,...,...,...
2095,652,river/river_000094.png,15,river
2096,1803,forest/forest_000141.png,7,forest
2097,1073,overpass/overpass_000227.png,13,overpass
2098,1125,chaparral/chaparral_000321.png,5,chaparral


In [15]:
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Use the InceptionV3 preprocessing function
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.5,
    height_shift_range=0.5,
    shear_range=0.3,
    zoom_range=0.5,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    brightness_range=[0.5, 1.5]
)
# Set the paths to your image directories
image_dir = "/kaggle/working/landuse-scene-classification/images"
train_test_val_dir = "/kaggle/working/landuse-scene-classification/images_train_test_val"

# Define the subdirectories for "train," "test," and "validation"
train_dir = os.path.join(train_test_val_dir, "train")
test_dir = os.path.join(train_test_val_dir, "test")
validation_dir = os.path.join(train_test_val_dir, "validation")

# Load the training data from "train" directory
image_data_train = datagen.flow_from_dataframe(dataframe = train_csv, directory = train_dir, x_col = "Filename", y_col = "ClassName",  target_size=(299,299), batch_size=32, class_mode="categorical", seed=seed)

# Load the testing data from "test" directory
image_data_test = datagen.flow_from_dataframe(dataframe = test_csv, directory = test_dir, x_col = "Filename", y_col = "ClassName", target_size=(299,299), batch_size=32, class_mode="categorical", seed=seed)

# Load the validation data from "validation" directory
image_data_val = datagen.flow_from_dataframe(dataframe = validation_csv, directory = validation_dir, x_col = "Filename", y_col = "ClassName", target_size=(299,299), batch_size=32, class_mode="categorical", seed=seed)


Found 7350 validated image filenames belonging to 21 classes.
Found 1050 validated image filenames belonging to 21 classes.
Found 2100 validated image filenames belonging to 21 classes.


In [18]:
!pip install tqdm
!pip install keras==3.4.1
import keras

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.5 MB/s eta 0:00:0000:01
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.


In [20]:
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import os

# Create directory for saving models if it doesn't exist
os.makedirs('models', exist_ok=True)

# Callback for saving the best model based on validation accuracy
checkpoint_best = ModelCheckpoint(
    'models/best_model.keras',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1,
)

# Callback for saving the latest model
checkpoint_latest = ModelCheckpoint(
    'models/latest_model.keras',
    monitor='val_accuracy',
    save_best_only=False,
    verbose=1,
)

# Callback for logging metrics to CSV
csv_logger = CSVLogger('cnn_fine_tuning_log.csv', append=True, separator=';')

In [21]:
from tensorflow.keras.callbacks import Callback
from tqdm.keras import TqdmCallback
# Train the model
history = model.fit(
    image_data_train,
    steps_per_epoch=32,
    validation_data=image_data_val,
    validation_steps=32 ,
    epochs=50,
    callbacks=[TqdmCallback(verbose=1), checkpoint_best, checkpoint_latest, csv_logger]
)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1722782045.713432     149 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1722782045.820767     149 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 898ms/step - accuracy: 0.1614 - f1_score: 0.0038 - loss: 2.9030 - precision: 0.3125 - recall: 0.0020

W0000 00:00:1722782083.669055     148 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



Epoch 1: val_accuracy improved from -inf to 0.34863, saving model to models/best_model.keras

Epoch 1: saving model to models/latest_model.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 169s 2s/step - accuracy: 0.1644 - f1_score: 0.0045 - loss: 2.8949 - precision: 0.3333 - recall: 0.0024 - val_accuracy: 0.3486 - val_f1_score: 0.1609 - val_loss: 2.1679 - val_precision: 0.7949 - val_recall: 0.0908
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 896ms/step - accuracy: 0.5171 - f1_score: 0.2158 - loss: 1.8768 - precision: 0.9976 - recall: 0.1253
Epoch 2: val_accuracy improved from 0.34863 to 0.61426, saving model to models/best_model.keras

Epoch 2: saving model to models/latest_model.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.5197 - f1_score: 0.2185 - loss: 1.8697 - precision: 0.9973 - recall: 0.1271 - val_accuracy: 0.6143 - val_f1_score: 0.4430 - val_loss: 1.3946 - val_precision: 0.9099 - val_recall: 0.2959
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 830ms/step - accuracy: 0.6948 - f1_score: 0.

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 3: saving model to models/latest_model.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.6951 - f1_score: 0.5318 - loss: 1.1700 - precision: 0.9545 - recall: 0.3729 - val_accuracy: 0.7692 - val_f1_score: 0.6433 - val_loss: 0.8336 - val_precision: 0.8438 - val_recall: 0.5192
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7975 - f1_score: 0.7109 - loss: 0.8404 - precision: 0.9247 - recall: 0.5807
Epoch 4: val_accuracy did not improve from 0.76923

Epoch 4: saving model to models/latest_model.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 96s 3s/step - accuracy: 0.7973 - f1_score: 0.7116 - loss: 0.8386 - precision: 0.9250 - recall: 0.5815 - val_accuracy: 0.7686 - val_f1_score: 0.7368 - val_loss: 0.7873 - val_precision: 0.8836 - val_recall: 0.6377
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step - accuracy: 0.7877 - f1_score: 0.7669 - loss: 0.7280 - precision: 0.9286 - recall: 0.6577
Epoch 5: val_accuracy improved from 0.76923 to 0.80957, saving model to models/best_mo

In [22]:
! pip install h5py


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [25]:
model.load_weights('/kaggle/working/models/best_model.keras')

In [28]:
from sklearn.metrics import classification_report, confusion_matrix
# Evaluate the model on the test dataset
test_loss, test_accuracy, test_recall, test_precision, test_f1 = model.evaluate(image_data_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
print(f'Test Recall: {test_recall}')
print(f'Test Precision: {test_precision}')
print(f'Test F1 Score: {test_f1}')

# Predict the labels for the test dataset
predictions = model.predict(image_data_test)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = image_data_test.classes
class_labels = list(image_data_test.class_indices.keys())

# Print classification report
from sklearn.metrics import classification_report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

# Generate confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print(conf_matrix)

# Save classification report to CSV
report_dict = classification_report(true_classes, predicted_classes, target_names=class_labels, output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv('classification_report.csv', index=True)

# Save confusion matrix to CSV
conf_matrix_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
conf_matrix_df.to_csv('confusion_matrix.csv', index=True)

33/33 ━━━━━━━━━━━━━━━━━━━━ 31s 930ms/step - accuracy: 0.9279 - f1_score: 0.9277 - loss: 0.2678 - precision: 0.9447 - recall: 0.9122
Test Loss: 0.2536562979221344
Test Accuracy: 0.9266666769981384
Test Recall: 0.907619059085846
Test Precision: 0.9463753700256348
Test F1 Score: 0.9263879656791687
33/33 ━━━━━━━━━━━━━━━━━━━━ 30s 914ms/step
                   precision    recall  f1-score   support

     agricultural       0.04      0.04      0.04        50
         airplane       0.04      0.04      0.04        50
  baseballdiamond       0.06      0.06      0.06        50
            beach       0.07      0.08      0.07        50
        buildings       0.05      0.06      0.06        50
        chaparral       0.00      0.00      0.00        50
 denseresidential       0.05      0.06      0.05        50
           forest       0.04      0.04      0.04        50
          freeway       0.00      0.00      0.00        50
       golfcourse       0.06      0.06      0.06        50
           h